In [1]:
import os

In [2]:
%pwd

'd:\\LoanTap\\LoanTap\\research'

In [3]:
os.chdir('../')

In [4]:
import pandas as pd

data = pd.read_csv("D:\\LoanTap\\LoanTap\\artifacts\\data_ingestion\\LoanTap.csv")

In [5]:
data.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,...,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,...,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,...,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,...,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,...,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210142 entries, 0 to 210141
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             210142 non-null  float64
 1   term                  210142 non-null  object 
 2   int_rate              210142 non-null  float64
 3   installment           210142 non-null  float64
 4   grade                 210142 non-null  object 
 5   sub_grade             210142 non-null  object 
 6   emp_title             198020 non-null  object 
 7   emp_length            200425 non-null  object 
 8   home_ownership        210142 non-null  object 
 9   annual_inc            210142 non-null  float64
 10  verification_status   210142 non-null  object 
 11  issue_d               210141 non-null  object 
 12  loan_status           210141 non-null  object 
 13  purpose               210141 non-null  object 
 14  title                 209213 non-null  object 
 15  

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE:str
    unzip_data_dir:Path
    all_schema:dict

In [8]:
from src.LoanTapPred.constants import *
from src.LoanTapPred.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config



In [9]:
import os
from src.LoanTapPred import logger

In [10]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema

            for col in all_cols:
                if col not in all_schema.keys():
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                
                elif data[col].dtype !=  all_schema[col]:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                        
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

                return validation_status
        except Exception as e:
            raise e
                


    

In [11]:
try: 
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-01-16 11:44:11,792: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-16 11:44:11,800: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-16 11:44:11,810: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-16 11:44:11,812: INFO: common: created directory at: artifacts]
[2025-01-16 11:44:11,813: INFO: common: created directory at: artifacts/data_validation]
